In [1]:
from __future__ import print_function
import numpy as np 
from numpy import random
import pandas as pd
import matplotlib.pyplot as plt 

import os
from os import makedirs
from os.path import exists, join

import keras
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import embeddings,Embedding,LSTM,GRU,Dense,Dropout,Conv1D, MaxPooling1D,Activation,Input, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.models import Model

In [3]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
!pip install snowballstemmer
!pip install gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jananisundaresan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

In [5]:
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [6]:
#read the data
df=pd.read_csv("~/Desktop/Thesis/train_fast.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13955 entries, 0 to 13954
Data columns (total 2 columns):
Genre       13955 non-null object
Synopsys    13955 non-null object
dtypes: object(2)
memory usage: 218.1+ KB


In [7]:
df = df.sample(frac=1)
df=df.iloc[:1000,]

In [8]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text


In [9]:
# apply the above function to df['text']
df['Synopsys'] = df['Synopsys'].map(lambda x: clean_text(x))

In [10]:
### Create sequence

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(df['Synopsys'])
vocab=len(tokenizer_obj.word_index)+1
vocab

16062

In [11]:
max_length=max([len(t.split()) for t in df['Synopsys']])
print("Maximum words in a sentence:"+ str(max_length))

Maximum words in a sentence:923


In [12]:
sequences=tokenizer_obj.texts_to_sequences(df['Synopsys'])

data=pad_sequences(sequences,maxlen=max_length,padding='post')

In [13]:
X=data
y=df['Genre']
print(X.shape)
print(y.shape)

X.shape

(1000, 923)
(1000,)


(1000, 923)

In [14]:
X[1].shape

(923,)

In [15]:
y = pd.get_dummies(df['Genre'].values)
print('Shape of label tensor:', X.shape)

Shape of label tensor: (1000, 923)


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(900, 923) (900, 10)
(100, 923) (100, 10)


In [18]:
with open(os.path.join('logs', 'metadata.tsv'), 'w') as f:
    np.savetxt(f, y)


In [19]:
tensorboard = TensorBoard(batch_size=256,
                          embeddings_freq=1,
                          embeddings_layer_names=['features'],
                          embeddings_metadata='metadata.tsv',
                          embeddings_data=X)


In [20]:

embeddings_index = dict()
f = open('Downloads/glove.840B.300d.txt',encoding='utf-8')
for line in f:
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 2196017 word vectors.


In [21]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, 300))
for word, index in tokenizer_obj.word_index.items():
    if index > vocab - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [23]:
from keras.layers import embeddings,LSTM,Dense,Dropout,GRU, SpatialDropout1D
import keras.optimizers as optimizers
from keras.regularizers import L1L2
embedding_layer = Embedding((vocab),300,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

input_sequence = Input(shape=(max_length,), dtype='int32')
embedded_sequence = embedding_layer(input_sequence)
bidirectional_lstm = Bidirectional(LSTM(150))(embedded_sequence)
preds = Dense(10, activation='softmax',name='features')(bidirectional_lstm)
model = Model(input_sequence, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])


model.summary()
model.fit(X_train, Y_train, callbacks=[tensorboard],
          epochs=30, batch_size=256)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 923)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 923, 300)          4818600   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 300)               541200    
_________________________________________________________________
features (Dense)             (None, 10)                3010      
Total params: 5,362,810
Trainable params: 544,210
Non-trainable params: 4,818,600
_________________________________________________________________
Epoch 1/30
900/900 [==============================] - 50s 55ms/step - loss: 2.2105 - acc: 0.2189
Epoch 2/30
900/900 [==============================] - 47s 52ms/step - loss: 1.8891 - acc: 0.3822
Epoch 3/30
900/900 [==============================] - 46s 52ms/s

In [24]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

100/100 [==============================] - 3s 29ms/step
Test set
  Loss: 1.619
  Accuracy: 0.580
